# Functions and Groupby: some additional work

In [42]:
import pandas as pd 
from pandas import DataFrame
import numpy as np
import datetime
import csv

## Read Data from csv

Change myfile to reflect location of your input file

In [43]:
myfile = 'P:/Ashok/Teaching/SemFinAnal/Data/5 stocks returns volume.csv'
stocks = pd.read_csv(myfile)
print(stocks.head(n=5))

   PERMNO      date TICKER          COMNAM      PRC       VOL       RET
0   10107  20070131   MSFT  MICROSOFT CORP  30.8600  13390496  0.033490
1   10107  20070228   MSFT  MICROSOFT CORP  28.1700  13007733 -0.083927
2   10107  20070330   MSFT  MICROSOFT CORP  27.8700  12832900 -0.010650
3   10107  20070430   MSFT  MICROSOFT CORP  29.9400   9676052  0.074273
4   10107  20070531   MSFT  MICROSOFT CORP  30.6901  13176580  0.028393


## Date manipulation

*Questions:*  
1. Can you obtain the month as a variable?

In [44]:
stocks['date'] = pd.to_datetime(stocks['date'],format='%Y%m%d' )
stocks['year'] = stocks['date'].dt.year
print(stocks.head(n=3))
stocks.dtypes

   PERMNO       date TICKER          COMNAM    PRC       VOL       RET  year
0   10107 2007-01-31   MSFT  MICROSOFT CORP  30.86  13390496  0.033490  2007
1   10107 2007-02-28   MSFT  MICROSOFT CORP  28.17  13007733 -0.083927  2007
2   10107 2007-03-30   MSFT  MICROSOFT CORP  27.87  12832900 -0.010650  2007


PERMNO             int64
date      datetime64[ns]
TICKER            object
COMNAM            object
PRC              float64
VOL                int64
RET              float64
year               int64
dtype: object

## Groupby & Applying functions on Groups



In [45]:
ret_grp = stocks.groupby(['TICKER','year'])

In [46]:
def get_annual_return(value):
    aret = 1
    for x in value:
        aret = aret * (x + 1)
    return aret - 1

#ret_data = stocks.groupby(['TICKER','year']).RET.apply(get_annual_return)
ret_data = ret_grp.RET.apply(get_annual_return)
ret_data.head(n=12)

TICKER  year
AAPL    2007    1.334749
        2008   -0.569114
        2009    1.469034
        2010    0.530665
        2011    0.255580
        2012    0.325217
        2013    0.080306
        2014    0.404335
        2015   -0.030401
DE      2007    0.986050
        2008   -0.580217
        2009    0.450359
Name: RET, dtype: float64

*Questions:*
1. What does the function calculate?
2. What compounding convention is used by this function? Can you think of an alternative?
3. Examine the structure of the code invoking the function. What is the sequence of the arguments?
4. Use dtypes on "ret_data" and also examine the index. Is there somthing interesting in the structure of the dataframe?
5. Can you use the above approach to find means of monthly returns (Jan, Feb, etc.) by stock? In other words, what is MSFT mean return for January, Feb, etc.?

In [47]:
ret_data.index

MultiIndex(levels=[['AAPL', 'DE', 'JNJ', 'MSFT', 'XOM'], [2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015]],
           labels=[[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4], [0, 1, 2, 3, 4, 5, 6, 7, 8, 0, 1, 2, 3, 4, 5, 6, 7, 8, 0, 1, 2, 3, 4, 5, 6, 7, 8, 0, 1, 2, 3, 4, 5, 6, 7, 8, 0, 1, 2, 3, 4, 5, 6, 7, 8]],
           names=['TICKER', 'year'])

## Another Example - get quarterly prices

In [48]:
# subset two stocks for 2007, also create quarter column
ticks = ("MSFT","AAPL")
stocks['quarter'] = (stocks['date'].dt.month-1)//3 + 1
stocks1 = stocks.loc[((stocks.year) == 2007) & ((stocks.TICKER).isin(ticks)), ['TICKER','PRC','year','date','quarter']]
print(stocks1.head(n=5))

  TICKER      PRC  year       date  quarter
0   MSFT  30.8600  2007 2007-01-31        1
1   MSFT  28.1700  2007 2007-02-28        1
2   MSFT  27.8700  2007 2007-03-30        1
3   MSFT  29.9400  2007 2007-04-30        2
4   MSFT  30.6901  2007 2007-05-31        2


In [49]:
# get the closing price for quarter: set group, define function, run function
ret_grp1 = stocks1.groupby(['TICKER','year','quarter'])

def get_last_price(value):
    lastprice = value[-1:]
    return lastprice

qterly_prc_data = ret_grp1.PRC.apply(get_last_price)
print(qterly_prc_data.head(n=5))

TICKER  year  quarter     
AAPL    2007  1        218     92.91
              2        221    122.04
              3        224    153.47
              4        227    198.08
MSFT    2007  1        2       27.87
Name: PRC, dtype: float64


In [50]:
# convert series with 4 indexes (3 names, 1 unnamed) to a dataframe with 4 indexes
newdf = qterly_prc_data.to_frame(name='price')
print(newdf.head(n=5))

                          price
TICKER year quarter            
AAPL   2007 1       218   92.91
            2       221  122.04
            3       224  153.47
            4       227  198.08
MSFT   2007 1       2     27.87


In [51]:
# convert the indexes to columns
newdf.reset_index(inplace=True)
print(newdf.head(n=5))

  TICKER  year  quarter  level_3   price
0   AAPL  2007        1      218   92.91
1   AAPL  2007        2      221  122.04
2   AAPL  2007        3      224  153.47
3   AAPL  2007        4      227  198.08
4   MSFT  2007        1        2   27.87


In [52]:
# delete unwanted column
newdf.drop('level_3', axis=1, inplace=True)
print(newdf.head(n=5))

  TICKER  year  quarter   price
0   AAPL  2007        1   92.91
1   AAPL  2007        2  122.04
2   AAPL  2007        3  153.47
3   AAPL  2007        4  198.08
4   MSFT  2007        1   27.87
